In [1]:
%matplotlib inline
import os
import numpy as np
from matplotlib import pyplot as plt
#
wd = os.getcwd()     # save current working directory
os.chdir('../src')   # change to directory containing our tools 
import simutils      # import necessary tools
import emio
os.chdir(wd)         # change back to directory containing sub.py

In [2]:
simulator_bin='~/gdrive_perso/Toolkit/Microscope_Simulator/TEM-simulator_1.3/src/TEM-simulator'

# Parameters

In [3]:
# Information about the molecular model
pdbdir   = 'pdbs/' # relative path to directory containing candidate PDB fiels
pdb      = '4v6x'  # ID of PDB file found in pdbdir
pdb_file = pdbdir+pdb+'.pdb'
# Information about the volume imaged
sample_dimensions = [1200,50,50] # [nm] diameter, thickness at center, thickness at edge.
# Information about the beam
electron_dose     = 1000           # [e/nm**2] dose per image
# Optimize the coverage of volume imaged with repeats of the molecular model
xy_range, numpart = simutils.define_grid(sample_dimensions,pdb_file=pdb_file,Dmax=40)
#
mrcdir = 'simulated_micrographs/'

Maximum dimension of the molecule: 40 nm
Grid of 200 molecules that cover the volume: -424 nm --> 424 nm (step size: 60 nm)


# Generator

In [4]:
#help(simutils.fill_parameters_dictionary)

In [6]:
for defoc in np.arange(2,2.5,0.5):
        pdb_file, mrc_file, crd_file, log_file, inp_file = emio.simio(pdbdir, pdb, pdb+'_copy1',
                                                                    mrcdir, pdb+'_defoc'+str(defoc))
        params_dictionary = simutils.fill_parameters_dictionary(mrc_file = mrc_file, 
                                                                pdb_file = pdb_file, voxel_size = 0.025, particle_mrcout = mrc_file, 
                                                                crd_file = crd_file, sample_dimensions = sample_dimensions,
                                                                dose = electron_dose, defocus = defoc, noise = 'no',
                                                                log_file = log_file)
        simutils.write_crd_file(numpart, xrange = xy_range, yrange = xy_range, crd_file = crd_file)
        simutils.write_inp_file(inp_file = inp_file, dict_params = params_dictionary)
        os.system('{0} {1}'.format(simulator_bin, inp_file))
        

simulated_micrographs/4v6x_copy1.txt already exists.
